In [26]:
import pandas as pd 

data_path = "data_annotated.csv"
data = pd.read_csv(data_path)
data.head()

,doc_id,url,cache,fulltext,nature,published,entity_name,entity_type,geo_path,extracted_text,embeddings,typed_embeddings,km_cluster_labels
0,1058/fb940_2023.020.pdf,https://www.ermont.fr/fileadmin/www.ermont.fr/...,https://datapolitics-public.s3.gra.io.cloud.ov...,https://datapolitics-public.s3.gra.io.cloud.ov...,acte.arrete,2023-02-17 00:00:00,Ermont,Commune,Ermont/CA Val Parisis/Val-d'Oise/Île-de-France,* REPUBLIQUE FRANÇAISE DEPARTEMENT du VAL D'OI...,[-5.00864536e-02 1.34651005e-01 -7.00120330e-...,"[-0.0500864536, 0.134651005, -0.070012033, 0.1...",1.0
1,6763/bb9f0990f8ed563f586d881c42df16d7df699606_...,https://www.correze.gouv.fr/contenu/telecharge...,https://datapolitics-public.s3.gra.io.cloud.ov...,https://datapolitics-public.s3.gra.io.cloud.ov...,acte.arrete,2023-01-13 00:00:00,Préfecture - Corrèze,Préfecture,Corrèze/Nouvelle-Aquitaine,PRÉFET DE LA CORRÈZE RECUEIL DES ACTES ADMINIS...,[-4.19478863e-02 1.81751396e-03 -1.92429498e-...,"[-0.0419478863, 0.00181751396, -0.0192429498, ...",0.0
2,6767/0ae0df0f3596ad4687621b7753a150fe55e75883_...,https://www.creuse.gouv.fr/contenu/telechargem...,https://datapolitics-public.s3.gra.io.cloud.ov...,https://datapolitics-public.s3.gra.io.cloud.ov...,acte.arrete,2023-01-16 00:00:00,Préfecture - Creuse,Préfecture,Creuse/Nouvelle-Aquitaine,PRÉFÈTE DE LA CREUSE RECUEIL DES ACTES ADMINIS...,[-3.28501053e-02 1.09066427e-01 -1.61550164e-...,"[-0.0328501053, 0.109066427, -0.161550164, 0.1...",0.0
3,2212/df590_CCPF_RSanc_approuve.pdf,https://www.cc-paysdefayence.fr/wp-content/upl...,https://datapolitics-public.s3.gra.io.cloud.ov...,https://datapolitics-public.s3.gra.io.cloud.ov...,acte.arrete,2023-03-01 00:00:00,CC du Pays de Fayence,Intercommunalité,CC du Pays de Fayence/Var/Provence-Alpes-Côte ...,Eau et Assainissement REGIE DES EAUX DU PAYS D...,[ 3.24184373e-02 -1.52655896e-02 -3.54909562e-...,"[0.0324184373, -0.0152655896, -0.0354909562, 0...",0.0
4,3395/bfdd09c728391ba9f29bf516a68553dedcd243cd_...,https://www.soisy-sous-montmorency.fr/wp-conte...,https://datapolitics-public.s3.gra.io.cloud.ov...,https://datapolitics-public.s3.gra.io.cloud.ov...,acte.arrete,2023-03-09 00:00:00,Soisy-sous-Montmorency,Commune,Soisy-sous-Montmorency/CA Plaine Vallée/Val-d'...,REPUBLIQUE FRANCAISE DECISION DU MAIRE PRISE L...,[-8.80177468e-02 -6.38791397e-02 1.20596491e-...,"[-0.0880177468, -0.0638791397, 0.120596491, 0....",1.0


In [27]:
data.columns

Index(['doc_id', 'url', 'cache', 'fulltext', 'nature', 'published',
       'entity_name', 'entity_type', 'geo_path', 'extracted_text',
       'embeddings', 'typed_embeddings', 'km_cluster_labels'],
      dtype='object')

In [28]:
# data.iloc[3]["url"]

In [29]:
data.shape

(14308, 13)

## Automatic Data Annotation with Transformers and Mixtral 7B

In [30]:
with open("hf-token.txt", "r") as f:
        HF_TOKEN = f.read()

In [31]:
from transformers import AutoTokenizer

model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_id, token=HF_TOKEN)

#### Prompt Functions WITH TRANSFORMERS

Important note: the API only supports a context length of 8k for this model, while the model may support 128k. Some random dude got around the problem by running a private endpoint and changing the 'Container Configuration', specifically the token settings to whatever length they required.

Apparent solution PAID: This part is relatively straightforward. Go to the the model card (e.g. https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct), Click on "Deploy" in the top right corner and select "Inference Endpoint". In the next page you can choose what hardware you want to run the model on, which will impact how much you will pay per hour. Set "Automatic Scale to Zero" to some value other than "never" to switch off the endpoint after X amount of time without request, so that you won't be paying for the endpoint while it's not in use. Then go to "Advanced Configuration" and set the maximum amount of tokens to whatever makes sense for your use case. With this procedure you will be able to make full use of the larger context windows of Llama 3 models.




In [32]:
def lazy_length_check(text, limit=8000, base_prompt_len=888):# 888 is the length of the current prompt

    chars = text.split(" ")
    len_text = len(chars)
    if len_text > limit - base_prompt_len: 
        print("Text over max. length. Applying padding.")
        padded_text = " ".join(chars[:limit - base_prompt_len])
        return padded_text 
    
    else:
        return text

In [33]:
def generate_prompt(text_to_classify):
    base_prompt = f"""
    <INSTRUCTIONS>
    Suivre ces étapes :
    1. Déterminez si le INPUT TEXT concerne un projet ou non.
    2. Si le texte concerne un projet, renvoyez 1. Si le INPUT TEXT ne concerne pas un projet, renvoyez 0.
    </INSTRUCTIONS>

    <FORMAT_DE_SORTIE>
    Il doit que votre réponse est un numéro unique: 0 ou 1.
    </FORMAT_DE_SORTIE>

    <EXEMPLES>
    1. INPUT TEXT: "Pas moins de 200 tableaux peints par les impressionnistes à Bougival ont été répertoriés. Nombreux furent les peintres à s'installer pour un temps à Bougival ou à poser leur chevalet sur les rives de la Seine pour immortaliser paysage et vie en bord du fleuve. Lebourg (Alfred) 1849 – 1928 : attiré par la vallée de la Seine, Albert Lebourg peint à de nombreuses reprises le quai qui borde la Seine venant de Rueil. Il peint Bord de la Seine à Bougival en 1885. Monet (Claude) 1840 – 1926 : il s'installe à Bougival en 1869. Avec son tableau La Seine à Bougival ou le Pont de Bougival peint vers 1869/1870, Claude Monet a représenté l’ancien pont de la ville qui était à péage. Morisot (Berthe) 1841 – 1895 : elle passe plusieurs étés à Bougival de 1881 à 1884 dans la maison qui va devenir, en 2024, l’espace muséal Berthe Morisot. Elle peint à Bougival une quarantaine de toiles : La Fable (1883), Le Quai à Bougival (1883), Eugène Manet et sa fille dans le jardin (1883), Dans la Véranda (1884), Jardin à Bougival (1884), Roses trémières (1884), etc. Pissarro (Camille) 1830 – 1903 : installé à Louveciennes puis à Pontoise, Pissarro peint plusieurs toiles à Bougival : Maisons à Bougival (1870), Bords de la Seine à Bougival (1871), La Route de Louveciennes (1872). Renoir (Auguste) 1849 – 1928 : Renoir n’a jamais séjourné à Bougival, contrairement à son ami Monet, mais"
    0

    2. INPUT TEXT: "AMBITIONS DE LA ZAC : Une réponse aux enjeux contemporains Construit sur une friche industrielle et au contact d’un futur parc et du grand paysage, ce nouveau quartier répond à diérents enjeux : > Sociétaux : • Répondre aux besoins en logements de la commune, • Proposer un quartier avec une densité dénie pour ralentir l’étalement urbain, • Orir un accès aux services, espaces de santé, commerces et transports en commun, • Desservir le quartier par le réseau de transports en commun de la métropole (prolongation de la ligne 55), • Conserver l’identité culturelle et patrimoniale des lieux grâce au maintien des éléments industriels du site. > Environnementaux : • Renforcer la biodiversité de ce terrain malmené depuis 60 ans, en intégrant des plantations résistantes aux fortes chaleurs et la perméabilisation du site à 50 %. • Dépolluer les sols, • Respecter la charte Écoquartier : obtension de la médaille d’argent QDO en phase conception (Quartier Durable d’Occitanie). • Construire des bâtiments respectant les normes environnementales en vigueur, • Favoriser les modes de transport alternatifs à la voiture : création de pistes cyclables et de cheminements doux, d’une centralité orant des commerces et des activités, un accès à moins de 10 minutes à pied à la rive droite de la commune, • Préserver la limite entre ville et plaine cultivée. • Protéger les spots de biodiversités identiés dans l’étude d’impact (environ 7000 m2 ), • Orir un espace public paysager qualitatif avec près d’1 ha de parc urbain et de venelles vertes."
    1

    <INPUT TEXT>
    {text_to_classify}
    </INPUT TEXT>
    """
    return base_prompt

In [34]:
system_msg = "Vous êtes un assistant. Votre tâche consiste à dire si un INPUT TEXT concerne un projet ou pas."

def format_prompt(prompt, system_msg=system_msg):
    messages = [{"role": "system", "content": system_msg},
                {"role": "user", "content": prompt},
                {"role": "assistant", "content": "Réponse:"}]
    messages_tokenized = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True, return_tensors="pt"
    )
    return messages_tokenized

In [35]:
generation_params = dict(
    temperature=0.2,
    repetition_penalty=1.0,
    do_sample=True,
    max_new_tokens=150,
    return_full_text=False,
    seed=42,
    max_time=None,
    stream=False,
    use_cache=False,
    wait_for_model=False,
)

#### Querying

In [36]:
import requests
import huggingface_hub

def query(payload=None, api_url=None):
    response = requests.post(api_url, headers=headers, json=payload)
    return response.json()

headers = {"Authorization": f"Bearer {huggingface_hub.get_token()}"}
api_url = "https://api-inference.huggingface.co/models/" + model_id

In [37]:
import re
# not used
def parse_response(output):
    
    regex_expression = r"[01]" # try parsing reasons in the future?
    regex = re.compile(regex_expression)
    match = re.search(regex, output)
    # print(match)
    parsed_response = match.group()
    # print(parsed_response)
    return parsed_response

In [41]:
def llm_annotate(text, verbose=True):
    
    text = lazy_length_check(text)
    
    prompt = generate_prompt(text)
    prompt_formated = format_prompt(prompt)
    
    output = query(payload={"inputs": prompt_formated, "parameters": {**generation_params}}, api_url=api_url)
    if verbose:
        print(output)
    
    # annotation = parse_response(output[0]["generated_text"])
    print(output[0]["generated_text"])
    
    return output[0]["generated_text"]

In [39]:
from tqdm import tqdm
tqdm.pandas()

data_non_na = data[data["mistral_annotation"].isna()] # filter by na to not re-annotate text

KeyError: 'mistral_annotation'

In [42]:
data["mistral_annotation"] = data["extracted_text"].apply(llm_annotate)

Text over max. length. Applying padding.
[{'generated_text': '\n\n0'}]


0
[{'generated_text': '\n\n    0'}]


    0
Text over max. length. Applying padding.
[{'generated_text': "\n\n    1. Pour l'INPUT TEXT 1, la réponse est 0, car le texte ne concerne pas un projet.\n    2. Pour l'INPUT TEXT 2, la réponse est 1, car le texte concerne un projet. Il s'agit d'un nouveau quartier qui répond à différents enjeux sociaux, environnementaux et de transport, et qui vise à conserver l'identité culturelle et patrimoniale des lieux."}]


    1. Pour l'INPUT TEXT 1, la réponse est 0, car le texte ne concerne pas un projet.
    2. Pour l'INPUT TEXT 2, la réponse est 1, car le texte concerne un projet. Il s'agit d'un nouveau quartier qui répond à différents enjeux sociaux, environnementaux et de transport, et qui vise à conserver l'identité culturelle et patrimoniale des lieux.
Text over max. length. Applying padding.
[{'generated_text': "\n\n1\n\nL'input text concerne un projet, il s'agit d'ambitio

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
# data.to_csv("data_annotated.csv", index=False)

## Data Annotation with OLlama and LANGCHAIN

In [1]:
import pandas as pd
from numpy import NaN

data = pd.read_csv("data_annotated.csv")


In [ ]:
# data["llama3:1b_llm_is_project"] = NaN

In [2]:
data.columns

Index(['doc_id', 'url', 'cache', 'fulltext', 'nature', 'published',
       'entity_name', 'entity_type', 'geo_path', 'extracted_text',
       'embeddings', 'typed_embeddings', 'km_cluster_labels',
       'llama3:1b_llm_is_project'],
      dtype='object')

In [3]:
unannotated_rows = data[data["llama3:1b_llm_is_project"].isna()].shape[0]
unannotated_rows

12125

In [4]:
from langchain_ollama import OllamaLLM
from langchain.prompts import PromptTemplate

llm = OllamaLLM(model="mistral:instruct") # 128k token window size

In [5]:
base_prompt ="""Does this text talk about a project?
INPUT TEXT:{text_to_classify}
If it is, answer 1. If it is not, answer 0.
"""

In [6]:
def llm_annotate_langchain(text_to_classify, base_prompt=base_prompt):
    
    prompt_template = PromptTemplate(input_variables=["text_to_classify"],
                                    template=base_prompt)
    prompt = prompt_template.format(text_to_classify = text_to_classify)
    output = llm.invoke(prompt)
    # annotation = parse_response(output)
    return output

In [ ]:
# example = data.iloc[9]["extracted_text"]
# output = llm_annotate_langchain(example, base_prompt)
# print(output)

1 (These are references to various administrative decisions and acts related to environmental permits, CITES applications, water and fishing police procedures, exceptions to certain wildlife protection articles, and the instruction of classified sites dossiers in the Nouvelle Aquitaine region. Additionally, there are multiple arrangements for setting the list of national days of public generosity appeal for the year 2023.)


In [7]:
def filter_na(df, col="llama3:1b_llm_is_project"):
    
    unannotated_data = df[df[col].isna()]
    unannotated_data_index = list(unannotated_data.index)
    
    return unannotated_data, unannotated_data_index

In [ ]:
def annotate(df, base_prompt, col="llama3:1b_llm_is_project", n_to_annotate=False):
    
    unannotated_data, unannotated_data_index = filter_na(df)
    
    n_left = len(unannotated_data_index)
    n_done = 0
    
    for index in unannotated_data_index:
        
        text = unannotated_data.at[index, "extracted_text"]
        df.at[index, col] = llm_annotate_langchain(text, base_prompt)
        
        n_left = n_left - 1
        n_done = n_done + 1
        
        print("Annotations left:", n_left)
        print("Annotations done:", n_done)
        
        if n_to_annotate != False: 
            print("To annotate:", n_to_annotate)
            
        print()
        
        if n_to_annotate != False and n_to_annotate <= n_done:
            return df
        
    return df

In [ ]:
_ , unannotated_data_index = filter_na(data)

while len(unannotated_data_index) > 0:
    data = annotate(data, base_prompt, n_to_annotate=100) # annotate and save in batches of X
    data.to_csv("data_annotated.csv", index=False)

Annotations left: 11608
Annotations done: 1
To annotate: 99

Annotations left: 11607
Annotations done: 2
To annotate: 99

Annotations left: 11606
Annotations done: 3
To annotate: 99

Annotations left: 11605
Annotations done: 4
To annotate: 99

Annotations left: 11604
Annotations done: 5
To annotate: 99

Annotations left: 11603
Annotations done: 6
To annotate: 99

Annotations left: 11602
Annotations done: 7
To annotate: 99

Annotations left: 11601
Annotations done: 8
To annotate: 99

Annotations left: 11600
Annotations done: 9
To annotate: 99

Annotations left: 11599
Annotations done: 10
To annotate: 99

Annotations left: 11598
Annotations done: 11
To annotate: 99

Annotations left: 11597
Annotations done: 12
To annotate: 99

Annotations left: 11596
Annotations done: 13
To annotate: 99

Annotations left: 11595
Annotations done: 14
To annotate: 99

Annotations left: 11594
Annotations done: 15
To annotate: 99

Annotations left: 11593
Annotations done: 16
To annotate: 99

Annotations left: